<a href="https://colab.research.google.com/github/rodrigocorvalan93/PM-Finance-Arg/blob/master/Rofex_implicitas_calculadora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table><thead><tr><th colspan="2"><img src="https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSb3TdeZOqFh5vvswFlggO2CZC1lyTZbCQvbQ&usqp=CAU" alt="Logo Delta Asset Managemetn" width="578" height="87"></th></tr></thead></table>
<h2>Tasas Impl&iacute;citas Rofex - Delta Asset Management</h2><p>Calculadora de curvas de tasas impl&iacute;citas.<br>Instrucciones:</p><p>1) Secci&oacute;n 1 - Instalaci&oacute;n de Paquetes - Conexi&oacute;n con demo account - Funciones Auxiliares (correrlo con play)</p><p>2) Secci&oacute;n 2 - Calculadora de tasas impl&iacute;citas (setear tipo de cambio y correr)</p>

# Instalación de Paquetes - Conexión con demo account - Funciones Auxiliares
---



In [ ]:
#@title Bloque Instalaciones
# %% Instalaciones
!pip install pyRofex
!pip install plotly
%load_ext google.colab.data_table

In [ ]:
#@title Bloque de importación de paquetes
# %% Imports
from ipywidgets import widgets
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import time
import pyRofex


In [ ]:
#@title Bloque de funciones auxiliares, token de Remarkets: { run: "auto" }
# %% Funciones auxiliares y token

def tea(precio, capital, plazo):
    """
    Calcula la TEA de una letra.
    Parámetros:
        precio  : Precio de la letra cada 100 VN.
        capital : Monto de capital que devuelve la letra cada 100 VN.
        plazo   : Plazo residual en días.
    """
    return (capital / precio) ** (365. / plazo) - 1.

def tna(precio, capital, plazo):
    """
    Calcula la TNA de una letra.
    Parámetros:
        precio  : Precio de la letra cada 100 VN.
        capital : Monto de capital que devuelve la letra cada 100 VN.
        plazo   : Plazo residual en días.
    """
    return ((capital / precio) - 1.) * (365. / plazo)
################# TOKENS REMARKETS ###########################
# Seteamos acceso:
usuario = "rodrigocorvalan934316"
contraseña = "negqnF0!"
cuenta = "REM4316"
# Ingresamos los parametros pare el entorno: REMARKET environment
pyRofex.initialize(user=usuario,
                   password=contraseña,
                   account=cuenta,
                   environment=pyRofex.Environment.REMARKET)

################# Código no muy necesario ###########################
# Setear curva de futuros:
instrumentos = ["DoOct20A", "DoNov20A", "DoDic20A", "DoEne21A", "DoEne21A",
                "DoFeb21A", "DoMar21A", "DoAbr21A", "DoMay21A", "DoJun21A",
                "DoJul21A", "DoAgo21A", "DoSep21A"]
# Obtengo lista de tickers de futuros:



In [ ]:
#@title Bloque A3500
# %% Bloque A3500
a3500 = pd.read_excel(
            'http://www.bcra.gob.ar/Pdfs/PublicacionesEstadisticas/com3500.xls',
            sheet_name='TCR diario y TCNPM', usecols="C,D", index_col=0,
            skiprows=4, header=None, parse_dates=True,
            names=['fecha','tca3500'])

f"El último dólar A3500: {a3500['tca3500'][-1]}"


'El último dólar A3500: 77.5317'

# Cotizaciones e implícitas Rofex

## Parámetros y Spot

In [ ]:
#@markdown FYI último A3500
f"El último dólar A3500: {a3500['tca3500'][-1]}"

'El último dólar A3500: 77.5317'

In [ ]:
#@title Bloque Impoertante: DATABASE (Actualizar bloque para actualizar todo) { run: "auto", vertical-output: true }
#@markdown Modificar Fx para cálculo de implícitas y elegir tipo de futuro
tipo_dolar_futuro = "Mayorista A: DOmmmaaA" #@param ["Mayorista A: DOmmmaaA", "Minorista: DOmmmaa"]
spot_fx =  77.5317#@param {type:"number"}
# %% Database
################# Código Necesario, database: ###########################
# Básicamente pido la lista de todos los instrumentos get_all_instruments()
# Y luego filtro aquellos que tengan 'DO' que son Dolar Futuro y como los
# Tickers de futuros se dividen en minoristas DOmmmyy y DOmmmyyA, pido
# Tickers con 7 caracteres y 8 caracteres para minoristaas y mayoristas resp.
pyRofex.get_market_data(ticker=instrumentos[2],
                        entries=[pyRofex.MarketDataEntry.LAST])
all = pyRofex.get_all_instruments()
dict_all_inst = all['instruments']
instrumento_inicial = all['instruments'][0]['instrumentId']['symbol']
fut_usd_rfx_A = [e['instrumentId']['symbol'] for e
                in all['instruments']
                if len(e['instrumentId']['symbol']) == 8
                    and 'DO' in e['instrumentId']['symbol']]
fut_usd_rfx = [e['instrumentId']['symbol'] for e
                in all['instruments']
                if len(e['instrumentId']['symbol']) == 7
                    and 'DO' in e['instrumentId']['symbol']]
# %% Convertidor de list dict to table para info de Especies rofex
# Initialise data to lists.
if tipo_dolar_futuro == "Mayorista A: DOmmmaaA":
  fut_usd_rfx_A
elif tipo_dolar_futuro == "Minorista: DOmmmaa":
  fut_usd_rfx_A = fut_usd_rfx

info = [pyRofex.get_instrument_details(futuros)['instrument'] for futuros
        in fut_usd_rfx_A]
# Creates DataFrame.
df = pd.DataFrame(info, index = fut_usd_rfx_A)
# Creamos id
df['id'] = [e['symbol'] for e in df['instrumentId']]
# Creamos columna de maturity con formato fecha
df['maturityDate'] = pd.to_datetime(df['maturityDate'], infer_datetime_format=True)
# Ordeno maturity date
df = df.sort_values('maturityDate')
# Creamos dias al vencimiento
df['DiasAlVto'] = (df['maturityDate'] - dt.datetime.now()).dt.days

# Creamos

mkt_data = [pyRofex.get_market_data(e)['marketData'] for e in fut_usd_rfx_A]
mkt_data_pd = pd.DataFrame(mkt_data, index = fut_usd_rfx_A)
df = pd.concat([df, mkt_data_pd], axis=1, join='inner')

# Formato Porcentajes:
format_dict = {
    'tea': '{:.2%}',
    'tna': '{:.2%}'
}





## Curva Implícita Settlement

In [ ]:
#@title Yield Curves a Settlement
curva_settlement = df[['id','DiasAlVto','SE']].dropna()
curva_settlement['precio'] = [e['price'] for e in curva_settlement['SE']]
curva_settlement['tna'] = tna(spot_fx,
                            curva_settlement['precio'],
                            curva_settlement['DiasAlVto'])
curva_settlement['tea'] = tna(spot_fx,
                            curva_settlement['precio'],
                            curva_settlement['DiasAlVto'])
# Cleaning curva_settlement

#Style pandas curva_settlement.style.hide_index().hide_columns(['SE']).format(format_dict)
#Style google:
curva_settlement.style.hide_index().hide_columns(['SE']).format(format_dict)

id,DiasAlVto,precio,tna,tea
DOOct20A,13,78.650000,41.66%,41.66%
DONov20A,44,81.540000,43.24%,43.24%
DODic20A,74,85.510000,50.98%,50.98%
DOEne21A,104,88.450000,49.59%,49.59%
DOFeb21A,132,92.030000,51.84%,51.84%
DOMar21A,165,96.050000,52.95%,52.95%
DOAbr21A,195,99.900000,54.10%,54.10%
DOMay21A,226,102.850000,52.83%,52.83%
DOJun21A,256,106.210000,52.82%,52.82%
DOAgo21A,317,113.870000,54.04%,54.04%


In [ ]:
#@title Gráfico Settlement
fig = px.scatter(curva_settlement, x='DiasAlVto', y="tna", trendline="lowess")
fig.show()

In [ ]:
#@title Resumen Sublots Rofex Settlement
fig = make_subplots(
    rows=1, cols=2,
    shared_xaxes=True,
    vertical_spacing=0.03,
    specs=[[{"type": "table"},{"type": "scatter"}]]
)
fig.add_trace(
    go.Scatter(
        x=curva_settlement["DiasAlVto"],
        y=curva_settlement["tna"],
        name="Rofex Settlement"
    ),
    row=1, col=2)

fig.add_trace(
    go.Table(
        header=dict(
          values=['Ticker','Días al Vto', 'Precio', 'TNA', 'TEA']),
        cells=dict(values=[
                    curva_settlement.id,
                    curva_settlement.DiasAlVto,
                    curva_settlement.precio,
                    curva_settlement.tna,
                    curva_settlement.tea
                    ],
                    format = [None,'.0f', '.2f','.2%', '.2%']
                   )
    ),
    row=1, col=1)

fig.update_layout(
    height=800,
    yaxis_tickformat = '%',
    showlegend=False,
    title_text="Curva Rofex Settlement (Ajuste)",
)

fig.show()